In [ ]:
from CoRe_Dataloader_ECSG import get_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
import math
import torchinfo
trainds = get_dataset(rsamp = .8)
testds = get_dataset(rsamp = .2)
train_dl = DataLoader(trainds,batch_size=6,shuffle = True,)
test_dl = DataLoader(testds,batch_size=6,shuffle = True,)

In [ ]:
class LDB(nn.Module):
    def __init__(self,inD,outD,dropout) -> None:
        super().__init__()
        inD = int(inD)
        outD = int(outD)
        self.seq = nn.Sequential(
            nn.Linear(inD,outD),
            nn.Dropout1d(p = dropout),
            nn.BatchNorm1d(outD)
        )
    def forward(self,x):
        return self.seq(x)


def conv_formula(d_in,kernel_size,padding = 0,dilation = 1,stride = 1):
    return math.floor(((d_in+(2*padding)-(dilation*(kernel_size-1))-1)/stride)+1)


class CDB(nn.Module):
    def __init__(self,x_in,y_in,in_channels:int,out_channels:int,kernel_size:int,stride = 1,padding = 0,dilation = 1,dropout = .2,) -> None:
        super().__init__()
        self.seq = nn.Sequential( # type: ignore
            nn.Conv2d(in_channels=in_channels,
                      out_channels= out_channels,
                      kernel_size=kernel_size,
                      stride = stride,
                      padding = padding,
                      dilation=dilation
                      ),
            nn.BatchNorm2d(out_channels),
            nn.Dropout2d(dropout),
        )
        self.x_out = conv_formula(x_in,kernel_size=kernel_size,stride=stride,padding=padding,dilation=dilation)
        self.y_out = conv_formula(y_in,kernel_size=kernel_size,stride=stride,padding=padding,dilation=dilation)
        self.out_channels = out_channels
    def forward(self,x):
        return self.seq(x)

def new_CDB(x_in,y_in,in_channels:int = 1,out_channels:int = 1,kernel_size:int = 3,stride = 1,padding = 0,dilation = 1,dropout = .2,device = "cpu"):
    cdb = CDB(x_in,y_in,in_channels,out_channels,kernel_size,stride,padding,dilation,dropout).to(device)
    return cdb,cdb.x_out,cdb.y_out,cdb.out_channels

In [ ]:
# problems: needs serious resizing to even work on this computer
class simple_ann_cog(nn.Module):
    def __init__(self,x,y,out_classes,device = "cpu") -> None:
        super().__init__()
        self.resize = Resize((200,200))
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(x*y,4096,dropout = .2).to(device)
        self.l2 = LDB(4096,4096/2,.2).to(device)
        self.l3 = LDB(4096/2,out_classes,.2).to(device)
        self.seq = nn.Sequential(self.l1,self.l2,self.l3)
        self.softmax = nn.Softmax(-1).to(device)
    def forward(self,x: torch.Tensor):
        ix = self.resize(x)
        xs = ix.shape
        ix = ix.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.flatten(ix)
        ix = self.seq(ix)
        return self.softmax(ix)

In [ ]:
# problems: needs serious resizing to even work on this computer
class single_cnn_ann_cog(nn.Module):
    def __init__(self,x,y,out_classes,device = "cpu") -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.conv1,xout,yout,out_channels = new_CDB(x_in = 400, y_in = 400,in_channels = 1, out_channels = 12,kernel_size = 7,stride = 3,dropout=.2,device=device)
        self.conv2,xout,yout,out_channels = new_CDB(x_in = xout,y_in=yout,in_channels=12,out_channels=32,kernel_size=5,stride=2,dropout=.2,device=device)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(xout*yout*out_channels,2048,dropout = .2).to(device)
        self.l2 = LDB(2048,1024,.2).to(device)
        self.l3 = LDB(1024,out_classes,.2).to(device)
        self.seq = nn.Sequential(self.flatten,self.l1,self.l2,self.l3)
        self.softmax = nn.Softmax(-1).to(device)
        print(xout,yout,out_channels)
    def forward(self,x: torch.Tensor):
        xs = x.shape
        ix = x.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.conv1(ix)
        ix = self.conv2(ix)
        ix = self.seq(ix)
        return self.softmax(ix)
model = single_cnn_ann_cog(400,400,19,"cuda:0")
torchinfo.summary(model,(1,400,400))

In [15]:
import wandb
wandb.init("sdisc-2d-2convlayer")
config = wandb.config
config.epochs = 100
config.inx = 400
config.iny = 400
optimizer = optim.Adam(params = model.parameters(),lr = 1e-3)
lossfn = nn.CrossEntropyLoss()

In [ ]:
l = new_accuracy(model,test_dl)